## Goal

Build a simple chain within our graph that uses 4 concepts/tools
- Use chat messages in the graph
- Invoking chat models within graph
- Binding tools to out LLM
- Executing tool calls in our graph

In [2]:
# general imports
import sys
import os, getpass
from openai import OpenAI
from utils import *
from IPython.display import Image
from pprint import pprint

# Import langgraph modules to create and visualize the graph
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage 
from typing_extensions import TypedDict
from typing import Literal

# Install the missing package
# %pip install langchain_anthropic
# %pip install langchain_groq
from langchain_groq import ChatGroq

In [3]:
# Let's make sure we have the API key set
GROQ_API_KEY = set_api_key('GROQ_API_KEY')

API key found in .env file for GROQ_API_KEY


#### Messages

Chat models can use `messages`, which captures different roles within a conversation (e.g., "system", "human", "assistant").
Messages also feature one or more content blocks that contain text or potentially multimodal data (e.g., images, audio, video).

<h5>Let's create a list of messages. Each message can be supplied with a few things:</h5>

- `content` - content of the message
- `name` - optionally, who is creating the message
-  `response_metadata` - optionally, a dict of metadata that is oftern specific to each model provider

In [4]:

messages = [AIMessage(content=f'So you wanted to know more starwars conception idea?', name = 'AI')]
messages.extend([HumanMessage(content=f'Yes, I am interested in starwars', name = 'Human')])
messages.extend([AIMessage(content=f'What do you want to know about starwars?', name = 'AI')])
messages.extend([HumanMessage(content=f'What is the starwars universe?', name = 'Human')])


In [5]:
for m in messages:
    m.pretty_print()

================================== Ai Message ==================================
Name: AI

So you wanted to know more starwars conception idea?
================================ Human Message =================================
Name: Human

Yes, I am interested in starwars
================================== Ai Message ==================================
Name: AI

What do you want to know about starwars?
================================ Human Message =================================
Name: Human

What is the starwars universe?


### Chat Models

Chat models use a sequence of message as input and support message roles.

There are many chat models available out there to choose from, in our case, we will use Groq.

We can load the AI model and invoke it with our list of messages.


In [6]:
ai = ChatGroq(model="mixtral-8x7b-32768", api_key=GROQ_API_KEY, temperature=0.8, max_retries=2)

In [7]:
result = ai.invoke(messages)
result.pretty_print()

================================== Ai Message ==================================

The Star Wars universe is a fictional galaxy far, far away that serves as the setting for the Star Wars franchise, which includes films, television shows, books, comics, and other media.

The Star Wars universe is home to a wide variety of planets, species, and cultures, as well as a rich history that spans thousands of years. It is a setting that is heavily influenced by science fiction, fantasy, and mythology, and it is known for its epic space battles, complex characters, and themes of good versus evil.

At the heart of the Star Wars universe is the struggle between the light side and the dark side of the Force, a metaphysical power that binds the galaxy together and gives certain individuals the ability to perform supernatural feats. The light side is associated with peace, justice, and selflessness, while the dark side is associated with power, aggression, and greed.

The Star Wars universe is a vast

In [8]:
result.response_metadata

{'token_usage': {'completion_tokens': 245,
  'prompt_tokens': 65,
  'total_tokens': 310,
  'completion_time': 0.380392248,
  'prompt_time': 0.008585825,
  'queue_time': 0.020265914000000003,
  'total_time': 0.388978073},
 'model_name': 'mixtral-8x7b-32768',
 'system_fingerprint': 'fp_c5f20b5bb1',
 'finish_reason': 'stop',
 'logprobs': None}

In [9]:
stream = ai.stream(messages)
full = next(stream)
for chunk in stream:
    full += chunk
full.pretty_print()

============================ Aimessagechunk Message ============================

The Star Wars universe is a fictional galaxy far, far away from our own, created by George Lucas. It is the setting for the Star Wars film franchise, which includes nine main saga films, as well as standalone movies, animated series, and a vast array of books, comics, and other media.

The universe is home to a wide variety of planets, species, and technologies, and is divided into several distinct eras, ranging from the ancient Old Republic to the current sequel trilogy. It is a place of epic battles, political intrigue, and personal struggles, as characters navigate the complex web of alliances, betrayals, and destinies that define the galaxy.

At its core, the Star Wars universe is a reflection of our own world, exploring themes of good vs. evil, the power of hope, and the importance of standing up for what is right. It has captivated audiences for decades and continues to inspire generations of fans.


<h4> Tools </h4>

Tools are an additional capability provided to LLMs that can be used to interact with outside world.

It could be leveraged to control different parts of the code, call an external API, interact with database, trigger a lambda function, etc.

Tools can be as simple as Python functions or as complex as integrating LLM with RAG for real time analysis.